In [1]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import re
import os
import spacy

2023-06-10 18:57:52.219320: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 18:57:52.264754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-10 18:57:52.265519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 18:57:56.821014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-10 18:58:02.344541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

In [2]:
def replace_multiple_whitespace_logic(match_group):
    if "\n" in match_group[0]:
        return "\n"
    else:
        return " "

def format_text(text):
    # remove multiple consecutve whitespaces or new lines
    return re.sub(r"[ \n]*\n[ \n]*|[ \t\r\f\v]{2, }", replace_multiple_whitespace_logic, text)

def process_html_page(url, timeout=2):
    r = req.get(url, timeout=timeout)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    raw_text = soup.get_text()
    trimmed_text = raw_text.strip(" \n")
    return format_text(trimmed_text)

In [3]:
furniture_pages_path = os.path.join("data", "furniture_stores_pages.csv")
furniture_pages = pd.read_csv(furniture_pages_path, header=0, names=["url"])
furniture_pages.head()

,url
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [4]:
furniture_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     704 non-null    object
dtypes: object(1)
memory usage: 5.6+ KB


In [5]:
data_path = os.path.join(os.getcwd(), "data", "raw")
text_data_path = os.path.join(data_path, "text")
tokenized_text_data_path = os.path.join(data_path, "tokenized_text.csv")

def create_data_dirs():
    os.makedirs(text_data_path)
    print(f"Create directory \"{text_data_path}\"")

def sample_furniture_pages(furniture_pages, num_samples):
    if furniture_pages.size > num_samples:
        sampled_furniture_pages = furniture_pages.sample(n=num_samples)
    else:
        sampled_furniture_pages = furniture_pages
    furniture_pages = furniture_pages.loc[~furniture_pages.index.isin(sampled_furniture_pages.index)]
    return sampled_furniture_pages, furniture_pages

def should_generate_new_data():
    return not os.path.isdir(text_data_path)

def generate_new_data(furniture_pages, num_samples, max_count_attempts):
    # try to sample num_samples but there might be many broken url in our samples so
    # it might be necessary to sample multiple times but no more than max_count_attempts
    print("Start web scarping to generate new data samples")
    create_data_dirs()
    count = 0
    count_attempts = 0
    count_total_samples = 0
    furniture_pages_copy = furniture_pages.copy()
    urls = []
    while (num_samples - count > 0 and
           count_attempts < max_count_attempts and
           not furniture_pages_copy.empty):
        sampled_furniture_pages, furniture_pages_copy = sample_furniture_pages(furniture_pages_copy, num_samples - count)
        count_total_samples += sampled_furniture_pages.size
        for _, url in sampled_furniture_pages.itertuples():
            try:
                text = process_html_page(url)
                urls.append(url)
                with open(os.path.join(text_data_path, f"{count + 1:03}.txt"), "w") as file:
                    file.write(text)
                    count += 1
            except req.RequestException as e:
                continue
        count_attempts += 1
    urls_df = pd.DataFrame({"url": pd.Series(urls)})
    urls_path_file = os.path.join(data_path, "sampled_urls.csv")
    urls_df.to_csv(urls_path_file)
                
    print(f"Sampled {count_total_samples}")
    print(f"{count} files were written in directory \"{text_data_path}\"")
    print(f"{count} files were tokenized and written in file \"{tokenized_text_data_path}\"")
    print(f"The urls used for samplling were written in file \"{urls_path_file}\"")

def flatten(list):
    result = []
    for sublist in list:
        for item in sublist:
            result.append(item)
    return result

def should_preprocess_data():
    return os.path.isdir(text_data_path) and not os.path.isfile(tokenized_text_data_path)

def tokenize_files(file_paths):
    nlp = spacy.load("en_core_web_sm")
    tokens = []
    for file_path in file_paths:
        with open(file_path) as file:
            text = file.read()
            doc = nlp(re.sub("\n", " ", text))
            file_tokens = [token.text for token in doc]
            if file_tokens:
                file_tokens.append("END_SEQUENCE")
                tokens.append(file_tokens)
    return flatten(tokens)
    
def preprocess_data():
    file_paths = [os.path.join(text_data_path, file) for file in os.listdir(text_data_path)]
    file_paths.sort()
    tokens = tokenize_files(file_paths)
    print(f"Read {len(file_paths)} files from directory \"{text_data_path}\"")
    df = pd.DataFrame({"word": tokens, "tag": np.nan})
    df.to_csv(tokenized_text_data_path)
    print(f"Tokenized data have been written in file \"{tokenized_text_data_path}\"")

def main():
    if should_generate_new_data():
        generate_new_data(furniture_pages, num_samples=100, max_count_attempts=8)
    if should_preprocess_data():
        preprocess_data()

## Expalation

When there isn't a directory "data/raw/text", try to sample 100 urls from "furniture_stores_pages.csv" and get those web pages. The processed html pages are written in the dirctory "data/raw/text". For each page is extracted the raw text where multiple consecutive white spaces and new lines are removed.

When there isn't a file "data/raw/tokenized_text.csv", read the files from the dirctory "data/raw/text" and tokenize them. The result is written in the file "data/raw/tokenized_text.csv". A file is considered a sequence of text whose ending is marked by the tag "END_SEQUENCE".

In [6]:
main()

Start web scarping to generate new data samples
Create directory "/home/eduard/residence/projects/ml/product_extraction/data/raw/text"
Sampled 217
98 files were written in directory "/home/eduard/residence/projects/ml/product_extraction/data/raw/text"
98 files were tokenized and written in file "/home/eduard/residence/projects/ml/product_extraction/data/raw/tokenized_text.csv"
The urls used for samplling were written in file "/home/eduard/residence/projects/ml/product_extraction/data/raw/sampled_urls.csv"
Read 98 files from directory "/home/eduard/residence/projects/ml/product_extraction/data/raw/text"
Tokenized data have been written in file "/home/eduard/residence/projects/ml/product_extraction/data/raw/tokenized_text.csv"
